## 1.文本格式数据 

In [2]:
import pandas as pd
# csv文件读入为DataFrame
df = pd.read_csv('examples1/testForcsvReading.csv')
df  

,name,password,age,major
0,Frank,1234,23,signal processing
1,x,stel,22,Data mining


In [7]:
# 使用read_table指定分割符
pd.read_table('examples1/testForcsvReading.csv', sep = '\t')

,"name,password,age,major"
0,"Frank,1234,23,signal processing"
1,"x,stel,22,Data mining"


### 对于无标题行的csv的读入方法
- pandas为其分配默认的列名;**header**
- 自定义列名**names**

In [13]:
df1 = pd.read_csv('examples/ex1.csv', header=None)
df1

,0,1,2,3,4
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,fool


In [14]:
df2 = pd.read_csv('examples/ex1.csv', names = ['a','b','c','d','message'])
df2

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,fool


### 将列指定为索引 index_col
- 将单个列指定为索引
- 多个已列表形式传入，层次化索引

In [15]:
# 传入单个索引
df3 = pd.read_csv('examples1/ex1.csv', names = ['a','b','c','d','message'], index_col = 'message')
df3

,a,b,c,d
message,,,,
hello,1,2,3,4
world,5,6,7,8
fool,9,10,11,12


In [16]:
# 层次化索引， 传入索引列表
df4 = pd.read_csv('examples1/ex2.csv', index_col = ['key1','key2'])
df4

value1  value2
key1 key2                
one  a          1       9
     b          2      10
     c          3      11
     d          4      12
two  a          5      13
     b          6      14
     c          7      15
     d          8      16

### 非固定分隔符传入正则表达式分割
对于有大量空格如\n的来说，read_table('examples\ex4.txt', sep='\s+')

### 其他参数如skiprows等 refer to table6-2

### 对缺失值的处理-用NA/NULL进行标识
na_values
- 单个
- 用字典对其进行表示

In [17]:
result = pd.read_csv('examples1/ex3.csv', na_values = ['NULL'])
result

,something,a,b,c,d,message
0,one,1,2,3.0,4,hello
1,two,5,6,NaN,8,NaN
2,three,9,10,11.0,12,fool


In [19]:
sentinels = {'message':['fool','NA'], 'something':['one']}
result = pd.read_csv('examples/ex3.csv', na_values = sentinels)
result

,something,a,b,c,d,message
0,NaN,1,2,3.0,4,hello
1,two,5,6,NaN,8,NaN
2,three,9,10,11.0,12,NaN


### 逐块读取文本
- 设置最大显示行数
- 读取前几行 nrows
- 逐块读取 chucksize-> 返回TextParser对象

In [24]:
# max_rows
pd.options.display.max_rows = 10
result = pd.read_csv('examples/ex6.csv')
result

,one,two,three,four,key
0,0.467976,-0.038649,-0.295344,-1.824726,L
1,-0.358893,1.404453,0.704965,-0.200638,B
2,-0.501840,0.659254,-0.421691,-0.057688,G
3,0.204886,1.074134,1.388361,-0.982404,R
4,0.354628,-0.133116,0.283763,-0.837063,Q
...,...,...,...,...,...
9995,2.311896,-0.417070,-1.409599,-0.515821,L
9996,-0.479893,-0.650419,0.745152,-0.646038,E
9997,0.523331,0.787112,0.486066,1.093156,K
9998,-0.362559,0.598894,-1.843201,0.887292,G


In [25]:
# nrows
result = pd.read_csv('examples/ex6.csv', nrows = 5)
result

,one,two,three,four,key
0,0.467976,-0.038649,-0.295344,-1.824726,L
1,-0.358893,1.404453,0.704965,-0.200638,B
2,-0.501840,0.659254,-0.421691,-0.057688,G
3,0.204886,1.074134,1.388361,-0.982404,R
4,0.354628,-0.133116,0.283763,-0.837063,Q


In [49]:
# chunkSize
chunker = pd.read_csv('examples/ex6.csv',chunksize = 1000)
chunker

In [50]:
tot = pd.Series([])
# 对key的出现次数进行统计
for piece in chunker:
    tot = tot.add(piece['key'].value_counts(), fill_value = 0)
    
tot = tot.sort_values(ascending = False)
tot[:26]

E    368.0
X    364.0
L    346.0
O    343.0
Q    340.0
     ...  
W    305.0
T    304.0
B    302.0
Z    288.0
C    286.0
Length: 26, dtype: float64

### 将数据写出到文本
- DataFrame对象，to_csv();sep指定分割符，缺失值指定na_rep， 行和列的标签指定index&heade（禁用）columns(以指定顺序排列)
- Series对象

In [52]:
dates = pd.date_range('1/1/2000', periods = 7)
ts = pd.Series(range(7), index = dates)
ts.to_csv('examples1/tseries.csv')

/home/esterl/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: FutureWarning: The signature of `Series.to_csv` was aligned to that of `DataFrame.to_csv`, and argument 'header' will change its default value from False to True: please pass an explicit value to suppress this warning.
  This is separate from the ipykernel package so we can avoid doing imports until


### 处理分隔符格式
- read_table()可对表格型数据处理
- python内置csv模块 可将文件型对象传给csv.reader,然后对reader对象进行迭代-> tuple
- 对数据进行整理： 字典构造式& zip
- csv.Dialect的定义新格式或直接提供csv关键字给csv.reader： 分割符、字符串引用规定refer to 6-3
- 手工处理分割符： csv.writer


In [56]:
cat examples/ex7.csv

"a","b","c"
"1","2","3"
"1","2","3"


In [59]:
import csv
f = open('examples/ex7.csv')
reader = csv.reader(f)
for line in reader:
    print(line)

['a', 'b', 'c']
['1', '2', '3']
['1', '2', '3']


In [71]:
# 整理数据
with open('examples/ex7.csv') as f:
    lines = list(csv.reader(f))
    
# 标题行& 数据行,整理为字典
header, values = lines[0],lines[1:]
data_dict = {h: v for h, v in zip(header, zip(*values))}
data_dict

{'a': ('1', '1'), 'b': ('2', '2'), 'c': ('3', '3')}

### JSON数据
Java Script Object Notation. HTTP请求在Web浏览器和其他应用程序之间发送数据的标准格式。
- Python库可以读写JSON数据。json.loads:JSON->Python
- json.dumps: Python对象->JSON格式

In [72]:
# json是比表格型文本如csv更为灵活的一种数据形式
import json
obj = """
{"name": "Wes",
 "places_lived": ["United States", "Spain", "Germany"],
 "pet": null,
 "siblings": [{"name": "Scott", "age": 30, "pets": ["Zeus", "Zuko"]},
              {"name": "Katie", "age": 38,
               "pets": ["Sixes", "Stache", "Cisco"]}]
}
"""
# python对象->Json
result = json.loads(obj)
result


{'name': 'Wes',
 'places_lived': ['United States', 'Spain', 'Germany'],
 'pet': None,
 'siblings': [{'name': 'Scott', 'age': 30, 'pets': ['Zeus', 'Zuko']},
  {'name': 'Katie', 'age': 38, 'pets': ['Sixes', 'Stache', 'Cisco']}]}

In [74]:
# json->python对象
asjson = json.dumps(result)
asjson

'{"name": "Wes", "places_lived": ["United States", "Spain", "Germany"], "pet": null, "siblings": [{"name": "Scott", "age": 30, "pets": ["Zeus", "Zuko"]}, {"name": "Katie", "age": 38, "pets": ["Sixes", "Stache", "Cisco"]}]}'

In [77]:
# Json->DataFrame,并选取字段子集
df = pd.DataFrame(result['siblings'], columns = ['name','age'])
df

,name,age
0,Scott,30
1,Katie,38


In [82]:
# pandas.read_json默认JSON数组的每个对象是表格的一行
data = pd.read_json('examples/example.json')
print('pandas->json\n',data)
# pandas->JSON
print('data->JSON\n', data.to_json())
# orient
print('orient to data\n', data.to_json(orient='records'))

pandas->json
    a  b  c
0  1  2  3
1  4  5  6
2  7  8  9
data->JSON
 {"a":{"0":1,"1":4,"2":7},"b":{"0":2,"1":5,"2":8},"c":{"0":3,"1":6,"2":9}}
orient to data
 [{"a":1,"b":2,"c":3},{"a":4,"b":5,"c":6},{"a":7,"b":8,"c":9}]


### XML和HTML： Web信息收集
Python有许多可以读写常见的HTML和XML格式数据的库，包括lxml、Beautiful Soup和html5lib。lxml的速度比较快，但其它的库处理有误的HTML或XML文件更好。
- pandas 的read_html方法对html


In [83]:
tables = pd.read_html('examples/fdic_failed_bank_list.html')
failures = tables[0]
failures.head()

,Bank Name,City,ST,CERT,Acquiring Institution,Closing Date,Updated Date
0,Allied Bank,Mulberry,AR,91,Today's Bank,"September 23, 2016","November 17, 2016"
1,The Woodbury Banking Company,Woodbury,GA,11297,United Bank,"August 19, 2016","November 17, 2016"
2,First CornerStone Bank,King of Prussia,PA,35312,First-Citizens Bank & Trust Company,"May 6, 2016","September 6, 2016"
3,Trust Company Bank,Memphis,TN,9956,The Bank of Fayette County,"April 29, 2016","September 6, 2016"
4,North Milwaukee State Bank,Milwaukee,WI,20364,First-Citizens Bank & Trust Company,"March 11, 2016","June 16, 2016"


In [86]:
# 数据清洗&分析
close_timestamps = pd.to_datetime(failures['Closing Date'])
close_timestamps.dt.year.value_counts()

2010    157
2009    140
2011     92
2012     51
2008     25
       ... 
2004      4
2001      4
2007      3
2003      3
2000      2
Name: Closing Date, Length: 15, dtype: int64

- lxml.objectify解析XML
Python有许多可以读写常见的HTML和XML格式数据的库，包括lxml、Beautiful Soup和html5lib。lxml的速度比较快，但其它的库处理有误的HTML或XML文件更好。

In [ ]:
# read more about XML

## 2.二进制格式数据

实现数据的高效二进制格式存储最简单的办法之一是使用Python内置的pickle序列化。**pandas对象**都有一个用于将数据以pickle格式保存到磁盘上的to_pickle方法 to_pickle('path statement')；读取pickle时采用pandas.read_pickle('path')

>注意：pickle仅建议用于短期存储格式。其原因是很难保证该格式永远是稳定的；今天pickle的对象可能无法被后续版本的库unpickle出来。虽然我尽力保证这种事情不会发生在pandas中，但是今后的某个时候说不定还是得“打破”该pickle格式。\

pandas内置支持**HDF5及MessagePack**两种二进制数据格式，建议多尝试以获得最适合你的分析工作的文件格式。其余有bcolz及Feather等


In [92]:
df = pd.read_csv('examples/ex1.csv')
print(df)
# DataFrame to pickle
df.to_pickle('examples/df_pickle')
# 读取pickle对象
print('read pickles\n', pd.read_pickle('examples/df_pickle'))

   a   b   c   d message
0  1   2   3   4   hello
1  5   6   7   8   world
2  9  10  11  12     foo
read pickles
    a   b   c   d message
0  1   2   3   4   hello
1  5   6   7   8   world
2  9  10  11  12     foo


### HDF5格式数据
HDF5是一种存储大规模科学数组数据的非常好的文件格式。它可以被作为C标准库，带有许多语言的接口，如Java、Python和MATLAB等。HDF5中的HDF指的是层次型数据格式（hierarchical data format）。每个HDF5文件都含有一个文件系统式的节点结构，它使你能够存储多个数据集并支持元数据。与其他简单格式相比，HDF5支持多种压缩器的即时压缩，还能更高效地存储重复模式数据。对于那些**非常大的无法直接放入内存的数据集，HDF5就是不错的选择，因为它可以高效地分块读写。**
>笔记：如果你要处理的数据位于远程服务器，比如Amazon S3或HDFS，使用专门为分布式存储（比如Apache Parquet）的二进制格式也许更加合适。Python的Parquet和其它存储格式还在不断的发展之中，所以这本书中没有涉及。

如果需要本地处理海量数据，我建议你好好研究一下PyTables和h5py，看看它们能满足你的哪些需求。。由于许多数据分析问题都是IO密集型（而不是CPU密集型），利用HDF5这样的工具能显著提升应用程序的效率。

>注意：HDF5不是数据库。它最适合用作“一次写多次读”的数据集。虽然数据可以在任何时候被添加到文件中，但如果同时发生多个写操作，文件就可能会被破坏。

In [119]:
# HDF5格式
import numpy as np
frame = pd.DataFrame({'a':np.random.randn(100)})

store = pd.HDFStore('mydata.h5')
store['obj1'] = frame
store['obj1_col'] = frame['a']
store

<class 'pandas.io.pytables.HDFStore'>
File path: mydata.h5

In [100]:
# HDF5文件中的对象类似字典可选取
store['obj1']

,a
0,-1.272719
1,1.742648
2,-0.758778
3,0.580737
4,0.413616
...,...
95,-0.872011
96,-1.154830
97,-1.487364
98,-0.352620


In [120]:
# HDFStore支持fixed和table两种存储格式。table更慢， 但支持特殊语法查询操作
# store['obj2'] = frame
store.put('obj2', frame, format = 'table')
store.select('obj2', where =['index >10 and index <30'])


,a
11,-1.083418
12,-1.162562
13,1.582263
14,0.113723
15,0.197886
...,...
25,-0.468973
26,0.668599
27,1.407684
28,-0.398569


In [121]:
store.close()

In [ ]:
# pandas.read_hdf函数可快捷使用上述工具
frame.to_hdf('mydata.h5', 'obj3', format = 'table')
pd.read_hdf('mydata.h5','obj3', where = ['index<15'])

### Microsoft Excel文件
excel->pandas
**pandas的ExcelFile类或pandas.read_excel函数**支持读取存储在Excel 2003（或更高版本）中的表格型数据
pandas->excel
ExcelWriter对象后pandas对象的to_excel()

In [124]:
#  ExcelFile类， 传递xlsx路径创造实例
# 一次读取多个表单 船舰Excelile类更好
xlsx = pd.ExcelFile('examples/ex1.xlsx')
pd.read_excel(xlsx, 'Sheet1')

,Unnamed: 0,a,b,c,d,message
0,0,1,2,3,4,hello
1,1,5,6,7,8,world
2,2,9,10,11,12,foo


In [125]:
# 直接将路径传入read_excel方法
frame = pd.read_excel('examples/ex1.xlsx','Sheet1')
frame

,Unnamed: 0,a,b,c,d,message
0,0,1,2,3,4,hello
1,1,5,6,7,8,world
2,2,9,10,11,12,foo


In [127]:
# pandas数据写入为excel
obj = pd.Series(np.random.randn(20))
# 创建ExcelWriter
writer = pd.ExcelWriter('examples/ex2.xlsx')
# pandas对象的to_excel方法写入
obj.to_excel(writer,'Sheet2')
writer.save()

## 3.Web APIs交互
requests包
完成liaoxuefeng web实战

## 4.数据库交互
    基于SQL的关系型数据库（如SQL Server、PostgreSQL和MySQL等）使用非常广泛，其它一些数据库也很流行。数据库的选择通常取决于性能、数据完整性以及应用程序的伸缩性需求。

In [142]:
import sqlite3

query ="""
CREATE TABLE test2
(a VARCHAR(20), b VARCHAR(20),
c REAL,  d INTEGER);
"""

con = sqlite3.connect('mydata.sqlite')
con.execute(query)
con.commit()

OperationalError: database is locked

In [141]:
data = [('Atlanta', 'Georgia', 1.25, 6),
            ('Tallahassee', 'Florida', 2.6, 3),
           ('Sacramento', 'California', 1.7, 5)]
stmt = "INSERT INTO test1 VALUES(?,?,?,?)"
con.executemany(stmt, data)

OperationalError: database is locked

In [134]:
# 从表中选取数据时，大部分Python SQL驱动器（PyODBC、psycopg2、MySQLdb、pymssql等）都会返回一个元组列表
curson = con.execute('select * from test1')
rows = curson.fetchall()
rows

[('Atlanta', 'Georgia', 1.25, 6),
 ('Tallahassee', 'Florida', 2.6, 3),
 ('Sacramento', 'California', 1.7, 5)]

In [135]:
# 将元祖列表传给DataFrame，对列名curson的description属性
curson.description

(('a', None, None, None, None, None, None),
 ('b', None, None, None, None, None, None),
 ('c', None, None, None, None, None, None),
 ('d', None, None, None, None, None, None))

In [137]:
pd.DataFrame(rows, columns = [x[0] for x in curson.description])

,a,b,c,d
0,Atlanta,Georgia,1.25,6
1,Tallahassee,Florida,2.60,3
2,Sacramento,California,1.70,5


这种数据规整操作相当多，你肯定不想每查一次数据库就重写一次。SQLAlchemy项目是一个流行的Python SQL工具，它抽象出了SQL数据库中的许多常见差异。pandas有一个read_sql函数，可以让你轻松的从SQLAlchemy连接读取数据。这里，我们用SQLAlchemy连接SQLite数据库，并从之前创建的表读取数据：

In [ ]:
import sqlalchemy as sqla

db = sqla.create_engine('sqlite:///mydata.sqlite')
pd.read_sql('select * from test1', db)